This is the notebook where I will implement the generator model.

In this notebook I will try different model to generate answers:

- bofenghuang/vigogne-2-13b-instruct : This to be a llama version but it is huge. 

- ClassCat/gpt2-base-french: the most used gpt model.

- moussaKam/barthez : which use the BART model suitable for generation tasks.

- JDBN/t5-base-fr-qg-fquad: which use T5 model.

Additional I I find time to annodate I will try BERT model to extract model.

In [ ]:
template = """
Given the following information, answer the question.

Context:
{% for document in documents %}
    {{ document.content }}
{% endfor %}

Question: {{question}}
Answer:
"""

In [ ]:
questions = ["Quand l’Ordonnance présidentielle a-t-elle été lue sur le plateau de la Radiotélévision nationale congolaise (RTNC)?",
             "Qui a été nommé pour remplacer Emmanuel Ramazani Shadary au poste de vice-Premier ministre et ministre de l’Intérieur et sécurité?",
             "Où et quand Henri Mova Sakanyi est-il né?",
             "Quelle est la carrière politique de Henri Mova Sakanyi en République démocratique du Congo?",
             "Quel est le poste actuel de Henri Mova Sakanyi au sein du Parti du peuple pour la Reconstruction et la Démocratie (PPRD)?"]

In [ ]:
model_id = "camembert-base"

In [ ]:
spacy_model = 'fr_core_news_md'

In [ ]:
from src.retriever.retrieval import HybridRetriever

In [ ]:
retriever = HybridRetriever(model_id=model_id, spacy_model=spacy_model)

In [ ]:
for question in questions:
    documents = retriever.run(question)
    print(f"Question: {question}")
    print("Answer:")
    for i, document in enumerate(documents):
        print(f"Document {i}: {document}")
    print("----" * 10)

### Generative Model part

In [ ]:
from transformers import T5ForConditionalGeneration, T5Tokenizer

In [ ]:
documents = retriever.run(questions[1])

In [ ]:
contexts = " ".join([document for document in documents])

In [ ]:
contexts

In [ ]:
generative_model_id = "JDBN/t5-base-fr-qg-fquad"
model = T5ForConditionalGeneration.from_pretrained(generative_model_id)
tokenizer = T5Tokenizer.from_pretrained(generative_model_id)

In [ ]:
input = f"question: {question} context: {contexts}"


In [ ]:
input

In [ ]:
encoded_input = tokenizer([input],
                          return_tensors='pt',
                          max_length=512,
                          truncation=True)
output = model.generate(input_ids=encoded_input.input_ids,
                        attention_mask=encoded_input.attention_mask,
                        do_sample=True,)
output = tokenizer.decode(output[0], skip_special_tokens=True)

In [ ]:
output

In [ ]:
random_seed = 42
from transformers import set_seed

set_seed(random_seed)

In [ ]:
for question in questions:
    documents = retriever.run(question)
    contexts = " ".join([document for document in documents])
    input_ = f"question: {question} context: {contexts}"
    encoded_input = tokenizer([input_],
                              return_tensors='pt')
    output = model.generate(input_ids=encoded_input.input_ids,
                            attention_mask=encoded_input.attention_mask,
                            do_sample=True,
                            top_k=5,
                            temperature=0.6)
    output = tokenizer.decode(output[0], skip_special_tokens=True)
    print(f"Question: {question}")
    print(f"Answer: {output}")
    print("----" * 10)

### Using Bert Model

In [ ]:
# Load model directly
bert_model_id = "AgentPublic/camembert-base-squadFR-fquad-piaf"


In [ ]:
from transformers import CamembertForQuestionAnswering, CamembertTokenizer

bert_tokenizer = CamembertTokenizer.from_pretrained(bert_model_id)
bert_model = CamembertForQuestionAnswering.from_pretrained(bert_model_id)

In [ ]:
import torch

In [ ]:
for question in questions:
    documents = retriever.run(question)
    contexts = " ".join([document for document in documents])
    input = f"question: {question} context: {contexts}"
    encoded_input = bert_tokenizer(input, return_tensors='pt', max_length=512, truncation=True)
    output = bert_model(**encoded_input)
    start_token_index = torch.argmax(output.start_logits)
    end_token_index = torch.argmax(output.end_logits)
    print(f"Question: {question}")
    answers_token = encoded_input['input_ids'][0][start_token_index:end_token_index+1]
    answer = bert_tokenizer.decode(answers_token)
    print(f"Answer: {answer}")
    print("----" * 10)

It is not got at generating response, I will come back and try BART and GPT2 and Alpaca.

Using Bart

In [ ]:
gpt_model_id = "ClassCat/gpt2-base-french"

In [ ]:
# Load model directly
from transformers import AutoModelForCausalLM, AutoTokenizer

gpt2_tokenizer = AutoTokenizer.from_pretrained("ClassCat/gpt2-base-french")
gpt2_model = AutoModelForCausalLM.from_pretrained("ClassCat/gpt2-base-french")

In [ ]:
for question in questions:
    documents = retriever.run(question)
    contexts = " ".join([document for document in documents])
    input_ = f"question: {question} context: {contexts}"
    encoded_input = gpt2_tokenizer([input_],
                              return_tensors='pt')
    output = gpt2_model.generate(input_ids=encoded_input.input_ids,
                            attention_mask=encoded_input.attention_mask,
                            do_sample=True,
                            top_k=5,
                            temperature=0.6,
                            max_new_tokens=30)
    output = gpt2_tokenizer.decode(output[0], skip_special_tokens=True)
    print(f"Question: {question}")
    answer = output.replace(input_, '')
    print(f"Answer: {answer}")
    print("----" * 10)

### Barthez

In [ ]:
# Load model directly
from transformers import MBartForConditionalGeneration

bartez_tokenizer = AutoTokenizer.from_pretrained("moussaKam/barthez")
bartez_model = MBartForConditionalGeneration.from_pretrained(
    "moussaKam/barthez")

In [ ]:
for question in questions:
    documents = retriever.run(question)
    contexts = " ".join([document for document in documents])
    input_ = f"question: {question} context: {contexts}"
    encoded_input = bartez_tokenizer([input_],
                              return_tensors='pt')
    output = bartez_model.generate(input_ids=encoded_input.input_ids,
                            attention_mask=encoded_input.attention_mask,
                            do_sample=True,
                            top_k=5,
                            temperature=0.6,)
    output = bartez_tokenizer.decode(output[0], skip_special_tokens=True)
    print(f"Question: {question}")
    answer = output.replace(input_, '')
    print(f"Answer: {answer}")
    print("----" * 10)

Out of all those model, the T5 model seems to give some promising results.

We will use it to generate questions and then use it to answer the question it generates.

https://www.deepset.ai/blog/generate-questions-automatically-for-faster-annotation